# <center> Идентификация пользователей по посещенным веб-страницам

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы делеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Данные собраны с прокси-серверов Университета Блеза Паскаля и имеют очень простой вид: <br>

<center>ID пользователя, timestamp, посещенный веб-сайт</center>

Скачать исходные данные можно по [ссылке](http://fc.isima.fr/~kahngi/cez13.zip) в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://yadi.sk/d/_HK76ZDo32AvNZ) на архив *capstone_websites_data.zip* (~7.1 Mb, в развернутом виде ~ 65 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных Вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге 10users лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога 150users – там 150 файлов;
 - В 3users_toy – игрушечный пример из 3 файлов, это для отладки кода предобработки, который Вы далее напишете.

На 5 неделе будет задание по [соревнованию](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt) Kaggle Inclass, которое организовано специально под Capstone проект нашей специализации. Соревнование уже открыто и, конечно, желающие могут начать уже сейчас.

# <center>Неделя 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы Scipy.sparse), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций 1 и 2 недели курса "Математика и Python для анализа данных":**
   - [Циклы, функции, генераторы, list comprehension](https://www.coursera.org/learn/mathematics-and-python/lecture/Kd7dL/tsikly-funktsii-ghienieratory-list-comprehension)
   - [Чтение данных из файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/8Xvwp/chtieniie-dannykh-iz-failov)
   - [Запись файлов, изменение файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/vde7k/zapis-failov-izmienieniie-failov)
   - [Pandas.DataFrame](https://www.coursera.org/learn/mathematics-and-python/lecture/rcjAW/pandas-data-frame)
   - [Pandas. Индексация и селекция](https://www.coursera.org/learn/mathematics-and-python/lecture/lsXAR/pandas-indieksatsiia-i-sieliektsiia)
   
**Кроме того, в задании будут использоваться библиотеки Python [glob](https://docs.python.org/3/library/glob.html), [pickle](https://docs.python.org/2/library/pickle.html) и класс [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html) из scipy.sparse.**

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark). Я использую Anaconda3 версии 4.2.0.

In [1]:
# pip install watermark
%load_ext watermark

In [2]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,scikit-learn -g

C:\Anaconda2\lib\site-packages\watermark-1.3.4-py2.7.egg\watermark\watermark.py:143: DeprecationWarning: Importing scikit-learn as `scikit-learn` has been depracated and will not be supported anymore in v1.7.0. Please use the package name `sklearn` instead.


CPython 2.7.12
IPython 4.2.0

numpy 1.11.3
scipy 0.18.1
pandas 0.19.2
matplotlib 1.5.1
statsmodels 0.8.0rc1
scikit-learn 0.18.1

compiler   : MSC v.1500 64 bit (AMD64)
system     : Windows
release    : 7
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit
Git hash   :


In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import pickle

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [4]:
user31_data = pd.read_csv('capstone_websites_data/10users/user0031.csv', 
                          header=None,
                  names=['id', 'timestamp', 'site'])

In [5]:
user31_data.head()

,id,timestamp,site
0,31,2013-11-15T08:17:10,api.dailymotion.com
1,31,2013-11-15T08:21:43,b12.myspace.com
2,31,2013-11-15T08:21:46,b12.myspace.com
3,31,2013-11-15T08:22:14,www.facebook.com
4,31,2013-11-15T08:37:30,pixel2368.everesttech.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user1.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">id</th>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user2.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">id</th>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">target</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов (используйте drop_duplicates, чтоб оставить только уникальные сессии), *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя. 
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

Детали:
- Смотрите чуть ниже пример вывода, что должна возвращать функция
- Используйте glob (или аналоги) для обхода файлов в каталоге
- Создайте частотный словарь уникальных сайтов (вида {'site_string': (site_id, site_freq)}) и заполняйте его по ходу чтения файлов. Начните с 1
- Рекомендуется меньшие индексы давать более часто попадающимся сайтам (приницип наименьшего описания)
- Не делайте entity recognition, считайте *google.com*, *http://www.google.com* и *www.google.com* разными сайтами (подключить entity recognition можно уже в рамках индивидуальной работы над проектом)
- Скорее всего в файле число записей не кратно числу *session_length*. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор [*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- Не оставляйте в частотном словаре сайт 0 (уже в конце, когда функция возвращает этот словарь)
- 150 файлов из *capstone_websites_data/150users/* у меня обработались примерно за 3 секунды, но многое, конечно, зависит от реализации функции и от используемого железа. И вообще, первая реализация скорее всего будет не самой эффективной, дальше можно заняться профилированием (особенно если планируете запускать этот код для 3000 пользователей)

In [34]:
from time import time
dict_add_time = 0;
def add_elem_to_dict(dictionary, element, filecount):
    global dict_add_time
    s_time = time()
    if element in dictionary:
        return filecount
    else:
        dictionary[element] = [filecount+1, 0]
        return filecount+1
    dict_add_time += time() - s_time
def add_freq_to_dict(dictionary, data):
    global dict_add_time
    s_time = time()
    for key in dictionary:
        a = dictionary[key][0]
        dictionary[key] = (a, data[a])
    dict_add_time += time() - s_time

In [35]:
def prepare_train_set(csv_files_mask, session_length=10):
    # Получаем список файлов, удовлетворяющих заданной маске
    site_dict = {}
    list_of_files = glob(csv_files_mask)
    data = []
    file_counter = 0
    for path in list_of_files:
        all_sites_from_file = []
        # Читаем файлы из списка
        with open(path, 'r') as fp:
            list_of_sites = []
            for line in fp:
                # Читаем строки из файла, каждую строку разбиваем на части
                list_of_tokens = line.strip().split(',')
                # Проверяем, что строка не пустая
                if len(list_of_tokens) != 3:
                    break
                # Добавляем элемент в словарь
                file_counter = add_elem_to_dict(site_dict, list_of_tokens[2], file_counter)
                all_sites_from_file.append(site_dict[list_of_tokens[2]][0])
                user_id = int(list_of_tokens[0])
        n = len(all_sites_from_file)
        ind = 0
        while True:
            if ind + 10 > n-1:
                data.append(all_sites_from_file[ind:n] + [0 for _ in range(10 - n + ind)] + [user_id])
                break
            data.append(all_sites_from_file[ind:ind+10] + [user_id])
            ind += 10
    df = pd.DataFrame(data,columns = ['site'+str(x) for x in range(1, session_length+1)] + ['target'])
    df = df.drop_duplicates()
    add_freq_to_dict(site_dict, np.unique(df.drop(['target'], axis = 1).values.ravel(), return_counts=True)[1])
    return df, site_dict

In [40]:
dataFrame, site_dict = prepare_train_set('capstone_websites_data/3users_toy/*', session_length=10)
dataFrame

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target
0,5,3,3,12,3,2,9,6,10,8,1
1,5,2,2,2,0,0,0,0,0,0,1
2,5,3,7,7,3,0,0,0,0,0,2
3,4,2,3,2,3,2,2,6,11,4,3
4,4,2,3,0,0,0,0,0,0,0,3
5,1,1,1,1,1,1,1,1,1,1,4
6,1,1,1,1,1,1,1,1,1,1,4


In [8]:
site_dict

{'accounts.google.com': (8, 1),
 'apis.google.com': (9, 1),
 'football.kulichki.ru': (6, 2),
 'geo.mozilla.org': (11, 1),
 'google.com': (1, 9),
 'mail.google.com': (5, 2),
 'meduza.io': (3, 3),
 'oracle.com': (2, 8),
 'plus.google.com': (7, 1),
 'vk.com': (4, 3),
 'yandex.ru': (10, 1)}

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [30]:
train_data_toy, site_freq_3users = prepare_train_set('capstone_websites_data/3users_toy/*', 
                                                     session_length=10)

In [31]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target
0,1,2,2,3,2,4,5,6,7,8,1
1,1,4,4,4,0,0,0,0,0,0,1
2,1,2,9,9,2,0,0,0,0,0,2
3,10,4,2,4,2,4,4,6,11,10,3
4,10,4,2,0,0,0,0,0,0,0,3


In [104]:
# частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой
site_freq_3users

{'accounts.google.com': (5, 1),
 'apis.google.com': (7, 1),
 'football.kulichki.ru': (9, 2),
 'geo.mozilla.org': (3, 1),
 'google.com': (4, 9),
 'mail.google.com': (6, 2),
 'meduza.io': (10, 3),
 'oracle.com': (2, 8),
 'plus.google.com': (8, 1),
 'vk.com': (1, 3),
 'yandex.ru': (11, 1)}

**1. Примените полученную функцию к данным по 10 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится (используйте pandas.DataFrame.drop_duplicates), запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к первому вопросу теста. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [10]:
from time import time

In [20]:
t_start = time()
dict_add_time = 0;
train_data_10users, site_freq_10users = \
    prepare_train_set('capstone_websites_data/10users/*', 
                      session_length=10)
print("Time elapsed: " + str(time()- t_start))
print(dict_add_time)
print(time()- t_start - dict_add_time)

Time elapsed: 0.263999938965
0.00200009346008
0.262999773026


In [21]:
train_data_10users.shape[0]

13084

In [13]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [14]:
write_answer_to_file(train_data_10users.shape[0], 
                     'answer1_1.txt')

**2. Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [15]:
len(site_freq_10users.keys())

4913

In [16]:
write_answer_to_file(len(site_freq_10users.keys()), 
                     'answer1_2.txt')

**3. Примените полученную функцию к данным по 150 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится (используйте pandas.DataFrame.drop_duplicates), запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к третьему вопросу. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [15]:
dict_add_time = 0;
t_start = time()
train_data_150users, site_freq_150users = \
    prepare_train_set('capstone_websites_data/150users/*', 
                      session_length=10)
print("Time elapsed: " + str(time() - t_start))
print(time()- t_start - dict_add_time)

Time elapsed: 6.83699989319
6.83800005913


In [19]:
train_data_150users.shape[0]

130786

In [20]:
len(site_freq_150users.keys())

27797

In [21]:
write_answer_to_file(train_data_150users.shape[0], 
                     'answer1_3.txt')

**4. Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [22]:
write_answer_to_file(len(site_freq_150users.keys()), 
                     'answer1_4.txt')

**5. Выведите топ-10 самых популярных сайтов среди посещенных 150 пользователями. Запишите ответ в файл *answer1_5.txt* через пробел – все 10 сайтов на одной строке.**

In [23]:
def itemgetter_own(*items):
    if len(items) == 1:
        item = items[0]
        def g(obj):
            return obj[item][1]
    return g

In [24]:
top10_popular = sorted(site_freq_150users.items(), key=itemgetter_own(1), reverse=True)  

In [25]:
top10_popular[:10]

[('www.google.fr', (1, 64449)),
 ('www.google.com', (3, 50875)),
 ('www.facebook.com', (41, 37080)),
 ('apis.google.com', (2, 29902)),
 ('s.youtube.com', (178, 25289)),
 ('clients1.google.com', (212, 24984)),
 ('mail.google.com', (28, 18769)),
 ('plus.google.com', (27, 18280)),
 ('safebrowsing-cache.google.com', (293, 17601)),
 ('twitter.com', (70, 15775))]

In [26]:
answer = reduce(lambda res, x: res + " " + x[0], top10_popular[:10], "")
answer

' www.google.fr www.google.com www.facebook.com apis.google.com s.youtube.com clients1.google.com mail.google.com plus.google.com safebrowsing-cache.google.com twitter.com'

In [27]:
write_answer_to_file(answer, 
                     'answer1_5.txt')

**Для дальнейшего анализа запишем полученные DataFrame в csv-файлы.**

In [28]:
train_data_10users.to_csv('capstone_websites_data/train_data_10users.csv', 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv('capstone_websites_data/train_data_150users.csv', 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

**Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. **

**Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.** 

In [32]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [33]:
def make_sparse_data(data):
    indptr = [0]
    indices = []
    sparse_data = []
    for row in data:
        val, cnt = np.unique(row[row != 0], return_counts=True)
        indptr.append(indptr[-1] + len(val))
        for v, c in zip(val, cnt):
            indices.append(v - 1)
            sparse_data.append(c)
    return np.uint64(sparse_data),  np.uint64(indices), np.uint64(indptr)

In [34]:
make_sparse_data(X_toy)

(array([1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 2, 4, 1, 2, 1, 1, 1, 1], dtype=uint64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  0,  3,  0,  1,  8,  1,  3,  5,  9,
        10,  1,  3,  9], dtype=uint64),
 array([ 0,  8, 10, 13, 18, 21], dtype=uint64))

In [35]:
X_toy

array([[ 1,  2,  2,  3,  2,  4,  5,  6,  7,  8],
       [ 1,  4,  4,  4,  0,  0,  0,  0,  0,  0],
       [ 1,  2,  9,  9,  2,  0,  0,  0,  0,  0],
       [10,  4,  2,  4,  2,  4,  4,  6, 11, 10],
       [10,  4,  2,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [36]:
data, indices, indptr = make_sparse_data(X_toy)
print(indptr)
print(indices)
print(data)
data

[ 0  8 10 13 18 21]
[ 0  1  2  3  4  5  6  7  0  3  0  1  8  1  3  5  9 10  1  3  9]
[1 3 1 1 1 1 1 1 1 3 1 2 2 2 4 1 2 1 1 1 1]


array([1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 2, 4, 1, 2, 1, 1, 1, 1], dtype=uint64)

In [37]:
X_sparse_toy = csr_matrix(make_sparse_data(X_toy))  

In [38]:
X_sparse_toy.todense()

matrix([[1, 3, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 2, 0, 4, 0, 1, 0, 0, 0, 2, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=uint64)

In [39]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [40]:
t_start = time()
X_sparse_150users = csr_matrix(make_sparse_data(X_150users))  
print("Time elapsed: %lf" % (time() - t_start))

Time elapsed: 2.727000


In [41]:
X_sparse_10users = csr_matrix(make_sparse_data(X_10users))  
X_sparse_150users = csr_matrix(make_sparse_data(X_150users)) 

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [42]:
with open('capstone_websites_data/X_sparse_10users.pkl', 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl)
with open('capstone_websites_data/y_10users.pkl', 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl)
with open('capstone_websites_data/X_sparse_150users.pkl', 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl)
with open('capstone_websites_data/y_150users.pkl', 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl)
with open('capstone_websites_data/site_freq_3users.pkl', 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl)
with open('capstone_websites_data/site_freq_10users.pkl', 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl)
with open('capstone_websites_data/site_freq_150users.pkl', 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl)

**Убедимся, что число столбцов в разреженных матрицах X_sparse_10users и X_sparse_150users равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно. Важно удостовериться, что потом мы все будем работать с одними и теми же данными.**

In [43]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [44]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

## Пути улучшения
В этом проекте свобода творчества на каждом шаге, а 7 неделя проекта посвящена общему описанию (.ipynb или pdf) и взаимному оцениванию проектов. Что еще можно добавить по первой части проекта:
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- Помимо явного создания разреженного формата можно еще составить выборки с помощью CountVectorizer, TfidfVectorizer и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt) Kaggle Inclass (желающие могут начать уже сейчас).

На следующей неделе мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 